In [1]:
# import of all libraries
import pandas as pd
from datetime import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import math
import geopy.distance
import geopy.geocoders
import folium
from folium import plugins
from folium.plugins import DualMap
from folium.plugins import MarkerCluster
%matplotlib inline

## Nacteni dat ze souboru CSV

In [2]:
bikes_data = pd.read_csv('edinburgh_bikes.csv')
weather_data = pd.read_csv('edinburgh_weather.csv')

## Prvni pruzkum dat
- pro zobecneny prehled jsem zobrazila jen prvnich pet a poslednich pet radku z obou dataframe
- zde vidime pocatecni i konecny index a take vsechny sloupce, ze kterych se dataframe sklada

In [3]:
bikes_data.head(3)

,index,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,1182,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653
1,1,2018-09-15 09:24:33,2018-09-15 09:41:09,995,259,St Andrew Square,North East corner,55.954749,-3.192774,262,Canonmills,near Tesco's,55.962804,-3.196284
2,2,2018-09-15 09:48:54,2018-09-15 10:46:40,3466,262,Canonmills,near Tesco's,55.962804,-3.196284,250,Victoria Quay,Entrance to Scottish Government Office,55.977638,-3.174116


In [4]:
bikes_data.tail(3)

,index,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
438256,12638,2021-06-30 23:49:03,2021-07-01 00:11:25,1342,256,St Andrews House,beside Jacobs ladder,55.953164,-3.181682,1091,Holyrood Road,Opposite St Leonards Land,55.949560,-3.180413
438257,12639,2021-06-30 23:49:03,2021-07-01 00:11:52,1369,256,St Andrews House,beside Jacobs ladder,55.953164,-3.181682,1091,Holyrood Road,Opposite St Leonards Land,55.949560,-3.180413
438258,12640,2021-06-30 23:58:33,2021-07-01 00:07:15,522,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,1769,Brunswick Place,Corner of Elm Row/Brunswick Street,55.960852,-3.180986


In [5]:
weather_data.head(3)

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
0,00:00,11 °c,11 °c,9 km/h from S,19 km/h,0.0 mm,79%,13%,1020 mb,Excellent,2018-09-01
1,03:00,13 °c,12 °c,11 km/h from SSW,19 km/h,0.0 mm,76%,96%,1020 mb,Excellent,2018-09-01
2,06:00,14 °c,13 °c,11 km/h from SSW,19 km/h,0.0 mm,84%,100%,1020 mb,Excellent,2018-09-01


In [6]:
weather_data.tail(3)

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
6333,15:00,10 °c,6 °c,42 km/h from SW,55 km/h,0.5 mm,75%,100%,989 mb,Excellent,2020-10-31
6334,18:00,9 °c,5 °c,38 km/h from WSW,54 km/h,0.0 mm,76%,79%,994 mb,Excellent,2020-10-31
6335,21:00,8 °c,4 °c,36 km/h from SW,49 km/h,0.1 mm,74%,79%,997 mb,Excellent,2020-10-31


## Uprava nazvu jednotlivych sloupcu pro naslednou snadnejsi orientaci 
- nektere sloupce maji velmi dlouhe nazvy, coz z meho pohledu neni uplne nejlepsi, proto si je prejmenuji

In [7]:
bikes_data.columns

Index(['index', 'started_at', 'ended_at', 'duration', 'start_station_id',
       'start_station_name', 'start_station_description',
       'start_station_latitude', 'start_station_longitude', 'end_station_id',
       'end_station_name', 'end_station_description', 'end_station_latitude',
       'end_station_longitude'],
      dtype='object')

In [8]:
weather_data.columns

Index(['time', 'temp', 'feels', 'wind', 'gust', 'rain', 'humidity', 'cloud',
       'pressure', 'vis', 'date'],
      dtype='object')

In [9]:
# renaming of columns for easier orientation
bikes_data['duration'] = round(bikes_data['duration']/60,2)
bikes_col_rename = {'duration':'duration in min',
                   'start_station_id':'start_id',
                   'start_station_name':'start_name',
                    'start_station_latitude':'s_lat',
                    'start_station_longitude':'s_lon',
                    'start_station_description':'description_start',
                    'end_station_id':'end_id',
                    'end_station_name':'end_name',
                    'end_station_latitude':'e_lat',
                    'end_station_longitude':'e_lon',
                    'end_station_description':'description_end'}
bikes_data.rename(bikes_col_rename,axis=1,inplace=True)

In [10]:
bikes_data.head(1)

,index,started_at,ended_at,duration in min,start_id,start_name,description_start,s_lat,s_lon,end_id,end_name,description_end,e_lat,e_lon
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,19.7,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,North East corner,55.954728,-3.192653


## Pruzkum dat
- datovy typ,
- velikost dat
- unikatni hodnoty
- zakladni descriptivni statistika pro numericke hodnoty

In [11]:
bikes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438259 entries, 0 to 438258
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              438259 non-null  int64  
 1   started_at         438259 non-null  object 
 2   ended_at           438259 non-null  object 
 3   duration in min    438259 non-null  float64
 4   start_id           438259 non-null  int64  
 5   start_name         438259 non-null  object 
 6   description_start  434118 non-null  object 
 7   s_lat              438259 non-null  float64
 8   s_lon              438259 non-null  float64
 9   end_id             438259 non-null  int64  
 10  end_name           438259 non-null  object 
 11  description_end    433570 non-null  object 
 12  e_lat              438259 non-null  float64
 13  e_lon              438259 non-null  float64
dtypes: float64(5), int64(3), object(6)
memory usage: 46.8+ MB


- Metoda info() nam poskytla zakladni informace o dataframe. Muzeme zde videt celkovy pocet radku i sloupcu, pocet nenulovych hodnot, datove typy jednotlivych sloupcu i velikost pameti, kterou data zabiraji.

- Lze si vsimnout, ze pocet start_id je jiny nez pocet start_name a stejne tak i u end_id a end_name. Lze to interpretovat tak, ze kazda stanice ma svoje unikatni ID(identifikacni cislo), ale jeji nazev se muze vyskytovat ve stejne podobe u jineho ID. V prubehu cele analyzi bude pro mne stezejni ID jednotlivych stanic a ne jejich jmena. 

In [12]:
bikes_data.nunique()  # unikatni hodnoty pro jednotlive sloupce

index                 31398
started_at           433059
ended_at             432562
duration in min       14032
start_id                198
start_name              169
description_start       208
s_lat                   221
s_lon                   221
end_id                  199
end_name                170
description_end         209
e_lat                   223
e_lon                   223
dtype: int64

In [13]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6336 entries, 0 to 6335
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   time      6336 non-null   object
 1   temp      6336 non-null   object
 2   feels     6336 non-null   object
 3   wind      6336 non-null   object
 4   gust      6336 non-null   object
 5   rain      6336 non-null   object
 6   humidity  6336 non-null   object
 7   cloud     6336 non-null   object
 8   pressure  6336 non-null   object
 9   vis       6336 non-null   object
 10  date      6336 non-null   object
dtypes: object(11)
memory usage: 544.6+ KB


- v dataframe weather mame vsechny hodnoty datoveho typu object

In [14]:
weather_data.nunique() # unique counts for columns

time          8
temp         31
feels        32
wind        581
gust         74
rain        106
humidity     60
cloud       101
pressure     88
vis           4
date        792
dtype: int64

## Uprava datovych typu
- ne vsechny hodnoty v dataframe maji spravny nebo vhodny datovy typ k tvorbe analyzy a proto upravime datovy typ podle nasi potreby. Ocenime to pri naslednych vypoctech nebo pri spojovani dat
- nejdrive prevedeme vsechny casove hodnoty, ktere jsou ve formatu object cili string na datovy type formatu datetime64, velmi to ocenime pri praci s casem

In [15]:
# convert data to another datatype in bikes_data
bikes_data['started_at'] = pd.to_datetime(bikes_data['started_at'])
bikes_data['ended_at'] = pd.to_datetime(bikes_data['ended_at'])

In [16]:
bikes_data.dtypes

index                         int64
started_at           datetime64[ns]
ended_at             datetime64[ns]
duration in min             float64
start_id                      int64
start_name                   object
description_start            object
s_lat                       float64
s_lon                       float64
end_id                        int64
end_name                     object
description_end              object
e_lat                       float64
e_lon                       float64
dtype: object

- do dataframe bikes_data pridame weekday, hours, week, month a year ktere ziskame z casoveho udaje. Pro tyto hodnoty pouziji start_at, casovy udaj pocatku vypujcky, ktery je pro mne vice vypovidajici nez udaj o konci.

In [17]:
bikes_data['date'] = bikes_data['started_at'].dt.date.astype('datetime64')
bikes_data['weekday'] = bikes_data['date'].dt.weekday
bikes_data['weekend'] = np.where(bikes_data['weekday'] >=5, 1, 0)
bikes_data['hour'] = bikes_data['started_at'].dt.hour
bikes_data['week'] = bikes_data['date'].dt.isocalendar().week
bikes_data['month'] = bikes_data['date'].dt.month
bikes_data['year'] = bikes_data['date'].dt.year

In [18]:
# season and weekend or weekday
season = (bikes_data['started_at'].dt.month-1)//3
season += (bikes_data['started_at'].dt.month == 3)&(bikes_data['started_at'].dt.day>=20)
season += (bikes_data['started_at'].dt.month == 6)&(bikes_data['started_at'].dt.day>=21)
season += (bikes_data['started_at'].dt.month == 9)&(bikes_data['started_at'].dt.day>=23)
season -= 3*((bikes_data['started_at'].dt.month == 12)&(bikes_data['started_at'].dt.day>=21)).astype(int)

bikes_data['season'] = season

In [19]:
bikes_data.head(1)

,index,started_at,ended_at,duration in min,start_id,start_name,description_start,s_lat,s_lon,end_id,...,e_lat,e_lon,date,weekday,weekend,hour,week,month,year,season
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,19.7,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,...,55.954728,-3.192653,2018-09-15,5,1,8,37,9,2018,2


- ve sloupci weekend (vikend) mame dve hodnoty 1 jako True a 0 False. Uz v prvnim radku vidime hodnotu 1, coz znamena ze tato jizda probehla o vikendu. Dale ve sloupci weekday mame hodnoty 0 az 6, ktere znaci dny v tydnu - 0 je pondeli a 6 je nedele. to nam pomaha se zorientovat na pracovni dny a vikendy.

In [20]:
weather_data['date'] = pd.to_datetime(weather_data['date'])

In [21]:
weather_data.dtypes

time                object
temp                object
feels               object
wind                object
gust                object
rain                object
humidity            object
cloud               object
pressure            object
vis                 object
date        datetime64[ns]
dtype: object

In [22]:
weather_data.head(1)

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
0,00:00,11 °c,11 °c,9 km/h from S,19 km/h,0.0 mm,79%,13%,1020 mb,Excellent,2018-09-01


- ve weather_data muzeme videt, ze vsechny sloupce jsou datoveho typu object (string) a hodnoty pro time jsou taky nevhodne. Proto pristoupime k uprave, kde vyuziji pomoc regulernich vyrazu a vsechny hodnoty, ktere budeme pouzivat pri numerickych vypoctech prevedun na cisla. Tabulku weather take upravim, protoze nebude potreba pouzit vsechny sloupce, ty ktere nepotrebujeme odstranime. Weather si pripravime tak, aby nam nim nebranilo v pohodlnem vyuziti v nasledujicich castech analyzy.

In [23]:
weather_data.temp.replace(to_replace='\D+',value='',regex=True,inplace=True)
weather_data.wind.replace(to_replace='\D+',value='',regex=True,inplace=True)
weather_data.rain.replace(to_replace='\D+',value='',regex=True,inplace=True)
weather_data.humidity.replace(to_replace='\D+',value='',regex=True,inplace=True)
weather_data.cloud.replace(to_replace='\D+',value='',regex=True,inplace=True)


weather_data['temp']=weather_data['temp'].astype('float')
weather_data['wind']=weather_data['wind'].astype('float')
weather_data['rain']=weather_data['rain'].astype('float')
weather_data['humidity']=weather_data['humidity'].astype('float')
weather_data['cloud']=weather_data['cloud'].astype('float')

In [24]:
weather_data.head(1)

,time,temp,feels,wind,gust,rain,humidity,cloud,pressure,vis,date
0,00:00,11.0,11 °c,9.0,19 km/h,0.0,79.0,13.0,1020 mb,Excellent,2018-09-01


In [25]:
# odstranime sloupce, ktere nejsou pro analyzu zasadni
weather_data.drop(['feels','gust','pressure'],axis=1,inplace=True)

- ted kdyz se podivame na prvni sloupec tabulky weather vidime, ze sloupce mame zmenene na datovy typ float, a jsou proto vhodne k vypoctum pro descriptivni statistiku

In [26]:
weather_data.head(1)

,time,temp,wind,rain,humidity,cloud,vis,date
0,00:00,11.0,9.0,0.0,79.0,13.0,Excellent,2018-09-01


- jeste upravime sloupec time do vhodnejsiho formatu

In [27]:
weather_data.time = pd.to_timedelta(weather_data['time'] + ':00')

In [28]:
weather_data.time

0      0 days 00:00:00
1      0 days 03:00:00
2      0 days 06:00:00
3      0 days 09:00:00
4      0 days 12:00:00
             ...      
6331   0 days 09:00:00
6332   0 days 12:00:00
6333   0 days 15:00:00
6334   0 days 18:00:00
6335   0 days 21:00:00
Name: time, Length: 6336, dtype: timedelta64[ns]

In [29]:
weather_data.head(9)

,time,temp,wind,rain,humidity,cloud,vis,date
0,0 days 00:00:00,11.0,9.0,0.0,79.0,13.0,Excellent,2018-09-01
1,0 days 03:00:00,13.0,11.0,0.0,76.0,96.0,Excellent,2018-09-01
2,0 days 06:00:00,14.0,11.0,0.0,84.0,100.0,Excellent,2018-09-01
3,0 days 09:00:00,14.0,14.0,1.0,88.0,78.0,Excellent,2018-09-01
4,0 days 12:00:00,16.0,15.0,0.0,87.0,87.0,Excellent,2018-09-01
5,0 days 15:00:00,17.0,14.0,0.0,83.0,97.0,Excellent,2018-09-01
6,0 days 18:00:00,16.0,12.0,0.0,88.0,98.0,Excellent,2018-09-01
7,0 days 21:00:00,14.0,11.0,0.0,94.0,52.0,Excellent,2018-09-01
8,0 days 00:00:00,13.0,10.0,0.0,95.0,31.0,Poor,2018-09-02


In [30]:
bikes_data.head(9)

,index,started_at,ended_at,duration in min,start_id,start_name,description_start,s_lat,s_lon,end_id,...,e_lat,e_lon,date,weekday,weekend,hour,week,month,year,season
0,0,2018-09-15 08:52:05,2018-09-15 09:11:48,19.70,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,...,55.954728,-3.192653,2018-09-15,5,1,8,37,9,2018,2
1,1,2018-09-15 09:24:33,2018-09-15 09:41:09,16.58,259,St Andrew Square,North East corner,55.954749,-3.192774,262,...,55.962804,-3.196284,2018-09-15,5,1,9,37,9,2018,2
2,2,2018-09-15 09:48:54,2018-09-15 10:46:40,57.77,262,Canonmills,near Tesco's,55.962804,-3.196284,250,...,55.977638,-3.174116,2018-09-15,5,1,9,37,9,2018,2
3,3,2018-09-16 12:01:36,2018-09-16 12:25:26,23.83,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,254,...,55.923479,-3.175385,2018-09-16,6,1,12,37,9,2018,2
4,4,2018-09-16 12:03:43,2018-09-16 12:11:16,7.53,255,Kings Buildings 4,X-Y Cafe,55.922001,-3.176902,253,...,55.923202,-3.171646,2018-09-16,6,1,12,37,9,2018,2
5,5,2018-09-16 12:13:56,2018-09-16 12:17:45,3.80,253,Kings Building 2,Sanderson Building,55.923202,-3.171646,252,...,55.924185,-3.173831,2018-09-16,6,1,12,37,9,2018,2
6,6,2018-09-16 12:19:19,2018-09-16 12:25:26,6.10,252,Kings Buildings 1,Grant Institute,55.924185,-3.173831,254,...,55.923479,-3.175385,2018-09-16,6,1,12,37,9,2018,2
7,7,2018-09-16 12:28:57,2018-09-16 12:47:12,18.23,254,Kings Building 3,Kings Building House,55.923479,-3.175385,264,...,55.939963,-3.171586,2018-09-16,6,1,12,37,9,2018,2
8,8,2018-09-16 12:29:18,2018-09-16 12:46:24,17.08,254,Kings Building 3,Kings Building House,55.923479,-3.175385,264,...,55.939963,-3.171586,2018-09-16,6,1,12,37,9,2018,2


In [31]:
weather_data.dtypes

time        timedelta64[ns]
temp                float64
wind                float64
rain                float64
humidity            float64
cloud               float64
vis                  object
date         datetime64[ns]
dtype: object

In [32]:
bikes_data.dtypes

index                         int64
started_at           datetime64[ns]
ended_at             datetime64[ns]
duration in min             float64
start_id                      int64
start_name                   object
description_start            object
s_lat                       float64
s_lon                       float64
end_id                        int64
end_name                     object
description_end              object
e_lat                       float64
e_lon                       float64
date                 datetime64[ns]
weekday                       int64
weekend                       int64
hour                          int64
week                         UInt32
month                         int64
year                          int64
season                        int64
dtype: object

- muzeme rict, ze timto mame obe dataframe pripraveny, prozkoumany a muzeme se pustit do samotne analyzy. 

## Zobrazeni pomoci Folium Map

- pri zobrazeni pomoci folium map vyuziji plugin pro dualMap, ktery mi zobrazi dve mapy vedle sebe. V jedne budou zobrazene pocatecni stanice = jejich pocet a poloha a v druhe konecne stanice = pocet a poloha. Dale pridam interaktivni mapu, kde budou stanice v hodinovem casovem intervalu.

In [33]:
dualMap = DualMap(location=[55.953251, -3.1882675],tiles = 'cartodbpositron',
                 zoom_start = 14)

In [34]:
# DataFrame Start Stations
s1 = bikes_data[['start_id','start_name','description_start','s_lat','s_lon']].value_counts()
s1 = pd.DataFrame(s1)
s1.reset_index(inplace=True)
s1.rename(columns={0:'Count'},inplace=True)

# DataFrame End Stations
e1 = bikes_data[['end_id','end_name','description_end','e_lat','e_lon']].value_counts()
e1 = pd.DataFrame(e1)
e1.reset_index(inplace=True)
e1.rename(columns={0:'Count'},inplace=True)

In [35]:
s1.drop_duplicates(subset='start_id',inplace=True)

In [36]:
s1.head(1)

,start_id,start_name,description_start,s_lat,s_lon,Count
0,265,Meadows East,Between Melville Terrace and Sciennes,55.939809,-3.182739,12104


In [37]:
e1.drop_duplicates(subset='end_id',inplace=True)

In [38]:
e1.head(1)

,end_id,end_name,description_end,e_lat,e_lon,Count
0,1728,Portobello - Kings Road,Foot of Kings Road next to the promenade,55.957915,-3.118332,16656


In [39]:
s1_locations = s1[['s_lat','s_lon']]
s1_list = s1_locations.values.tolist()
len(s1_list) # delka listu tvoreneho z unikatnich hodnot langtitude a longtitude pro start_stations

197

In [40]:
e1_locations = e1[['e_lat','e_lon']]
e1_list = e1_locations.values.tolist()
len(e1_list) # delka listu tvoreneho z unikatnich hodnot langtitude a longtitude pro end_stations

198

In [41]:
# tvorba markercluster pro start stations
marker_cluster = MarkerCluster().add_to(dualMap.m1)

for i, row in s1.iterrows():
    lat = s1.at[i,'s_lat']
    lon = s1.at[i,'s_lon']
    
    popup = 'Id: ' + str(s1.at[i,'start_id']) + '<br>' + \
          str(s1.at[i, 'start_name']) + '<br>' + \
          'Description: ' + s1.at[i,'description_start'] + '<br>' + \
          '<br>' + 'Counts: ' + str(s1.at[i,'Count'])
    
    folium.Marker(location=[lat,lon], popup=popup,
                  icon=folium.Icon(color='darkblue',
                                    icon_color='white',
                                    icon='bicycle',
                                    angle=0,
                                    prefix='fa')).add_to(marker_cluster)


In [42]:
# tvorba markerclusteru pro end_stations
markerCluster = MarkerCluster().add_to(dualMap.m2)

for i, row in e1.iterrows():
    lat = e1.at[i,'e_lat']
    lon = e1.at[i,'e_lon']
    
    popup = 'Id: ' + str(e1.at[i,'end_id']) + '<br>' + \
          str(e1.at[i, 'end_name']) + '<br>' + \
          'Description: ' + e1.at[i,'description_end'] + '<br>' + \
          '<br>' + 'Counts: ' + str(e1.at[i,'Count'])
    
    folium.Marker(location=[lat,lon], popup=popup, 
                 icon = folium.Icon(color='red',
                             icon_color='white',
                             icon='bicycle',
                            angle=0,
                            prefic='fa')).add_to(markerCluster)

In [43]:
dualMap

In [44]:
# saving map
dualMap.save('DualMap-start and end stations, Id, Counts and descriptions .html')

- ve vyse uvedene mape mame zobrazene stanice i pocet stanic v jednotlivych castech Edinburgu. V leve casti jsou to startovni (vychozi) stanice a v prave konecna stanice. Pokud rozklikneme na konkretni stanici, zobrazi se Id stanice, popis stanice a jeji jmeno a take cislo, urcujici, kolikrat byla stanice v prubehu existence vyuzita. Stejne tak muzeme videt pocet stanic nachazejicich se v jednotlivych oblastech Edinburghu.

In [45]:
bikes = bikes_data.copy()
bikes.set_index('started_at',inplace=True)

In [46]:
bikes.head(1)

,index,ended_at,duration in min,start_id,start_name,description_start,s_lat,s_lon,end_id,end_name,...,e_lat,e_lon,date,weekday,weekend,hour,week,month,year,season
started_at,,,,,,,,,,,,,,,,,,,,,
2018-09-15 08:52:05,0,2018-09-15 09:11:48,19.7,247,Charlotte Square,North Corner of Charlotte Square,55.952335,-3.207101,259,St Andrew Square,...,55.954728,-3.192653,2018-09-15,5,1,8,37,9,2018,2


In [47]:
# help us pivoting
bikes['type'] = 'stations'

In [48]:
# aggregate number of trips for each start station by hour of the day
start = bikes.pivot_table('duration in min',
                        index=['start_id',
                               'date',
                              's_lat',
                              's_lon',
                              bikes.index.hour],
                        columns='type',
                        aggfunc='count').reset_index()
start

type,start_id,date,s_lat,s_lon,started_at,stations
0,171,2018-09-17,55.943084,-3.188311,10,3
1,171,2018-09-17,55.943084,-3.188311,13,1
2,171,2018-09-17,55.943084,-3.188311,14,4
3,171,2018-09-17,55.943084,-3.188311,15,9
4,171,2018-09-17,55.943084,-3.188311,16,8
...,...,...,...,...,...,...
273355,2268,2021-06-30,55.956535,-3.186248,16,1
273356,2268,2021-06-30,55.956535,-3.186248,17,2
273357,2268,2021-06-30,55.956535,-3.186248,18,1
273358,2268,2021-06-30,55.956535,-3.186248,20,1


In [49]:
start.columns = ['start_id','date','lat','lon','hour','stations']

# define color
start['fillColor'] = '#55aa7f'

# Stop with less than one daily ride
# different color
start.loc[start['stations']<1, 'fillColor'] = '#ff0000'

start

,start_id,date,lat,lon,hour,stations,fillColor
0,171,2018-09-17,55.943084,-3.188311,10,3,#55aa7f
1,171,2018-09-17,55.943084,-3.188311,13,1,#55aa7f
2,171,2018-09-17,55.943084,-3.188311,14,4,#55aa7f
3,171,2018-09-17,55.943084,-3.188311,15,9,#55aa7f
4,171,2018-09-17,55.943084,-3.188311,16,8,#55aa7f
...,...,...,...,...,...,...,...
273355,2268,2021-06-30,55.956535,-3.186248,16,1,#55aa7f
273356,2268,2021-06-30,55.956535,-3.186248,17,2,#55aa7f
273357,2268,2021-06-30,55.956535,-3.186248,18,1,#55aa7f
273358,2268,2021-06-30,55.956535,-3.186248,20,1,#55aa7f


- V horni pomocne dataframe mame upravenou puvodni dataframe tak, ze muzeme videt, kolikrat byla vyuzita konkretni pocatecni stanice v jednotlivych hodinach v prubehu kazdeho dne. Pokud byla stanice vyuzita min jak dvakrat mame ji oznacenou jinou barvou nez pokud byla vyuzita vickrat. Odvodila jsem to od toho, ze prumer vychazel 2 krat provedena vypujcka pro kazdou stanici v hodine.

In [50]:
e_map = bikes.reset_index().set_index('ended_at')

In [51]:
# aggregate number of trips for each start station by hour of the day
end = e_map.pivot_table('duration in min',
                        index=['end_id',
                               'date',
                              'e_lat',
                              'e_lon',
                              bikes.index.hour],
                        columns='type',
                        aggfunc='count').reset_index()

end

type,end_id,date,e_lat,e_lon,started_at,stations
0,171,2018-09-17,55.943084,-3.188311,9,1
1,171,2018-09-17,55.943084,-3.188311,11,1
2,171,2018-09-17,55.943084,-3.188311,12,1
3,171,2018-09-17,55.943084,-3.188311,13,3
4,171,2018-09-17,55.943084,-3.188311,14,3
...,...,...,...,...,...,...
272778,2268,2021-06-30,55.956535,-3.186248,11,4
272779,2268,2021-06-30,55.956535,-3.186248,12,1
272780,2268,2021-06-30,55.956535,-3.186248,15,1
272781,2268,2021-06-30,55.956535,-3.186248,18,2


In [52]:
end.columns = ['end_id','date','lat','lon','hour','stations']
end['fillColor'] = '#00007f'
end.loc[end['stations']<1,'fillColor'] = '#ffa10b'

end

,end_id,date,lat,lon,hour,stations,fillColor
0,171,2018-09-17,55.943084,-3.188311,9,1,#00007f
1,171,2018-09-17,55.943084,-3.188311,11,1,#00007f
2,171,2018-09-17,55.943084,-3.188311,12,1,#00007f
3,171,2018-09-17,55.943084,-3.188311,13,3,#00007f
4,171,2018-09-17,55.943084,-3.188311,14,3,#00007f
...,...,...,...,...,...,...,...
272778,2268,2021-06-30,55.956535,-3.186248,11,4,#00007f
272779,2268,2021-06-30,55.956535,-3.186248,12,1,#00007f
272780,2268,2021-06-30,55.956535,-3.186248,15,1,#00007f
272781,2268,2021-06-30,55.956535,-3.186248,18,2,#00007f


In [53]:
def create_data(df):
    data = []
    
    for _,row in df.iterrows(): # iterace skrz radky
        d = {                   # tvorba d 
            'type':'Feature',
            'geometry':{        # tvorba geometry point,vezme lat a lon z dataframe
                'type':'Point',
                'coordinates':[row['lon'],row['lat']]
            },
            'properties': {   # definujeme vlastnosti dalsich promennych
                'time':pd.to_datetime(row['hour'],unit='h').__str__(),
                'style':{'color':''},
                'icon':'circle',
                'iconstyle':{
                    'fillColor':row['fillColor'],
                    'fillOpacity': 0.8,
                    'stroke':'true',
                    'radius': row['stations'] + 5
                }
            }
        }
        data.append(d)
    return data


In [54]:
start_geojson = create_data(start)

In [55]:
start_geojson[0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-3.188311072722399, 55.94308406301113]},
 'properties': {'time': '1970-01-01 10:00:00',
  'style': {'color': ''},
  'icon': 'circle',
  'iconstyle': {'fillColor': '#55aa7f',
   'fillOpacity': 0.8,
   'stroke': 'true',
   'radius': 8}}}

In [ ]:
end_geojson = create_data(end)

In [ ]:
from folium.plugins import TimestampedGeoJson

dualMap2 = DualMap(location=[55.953251, -3.1882675],tiles = 'cartodbpositron',
                 zoom_start = 14)

TimestampedGeoJson(start_geojson,
                  period = 'PT1H',
                  duration = 'PT1M',
                  transition_time = 1000,
                  auto_play = True).add_to(dualMap2.m1)
TimestampedGeoJson(end_geojson,
                  period = 'PT1H',
                  duration = 'PT1M',
                  transition_time = 1000,
                  auto_play = True).add_to(dualMap2.m2)
dualMap2

In [ ]:
dualMap2.save('DualMap-interactive map over hours.html')

- ve vyse zobrazenem interaktivnim grafu vidime vyvoj vypujcek v prubehu denni doby. V leve casti jsou pocatecni stanice a v prave jsou konecne stanice.

# Distance
- spocitame vzdalenosti a vytvorime novy sloupce k bikes_data, kde budou spocteny vzdalenosti mezi jednotlivymi stanicemi v Km. K vypoctu vzdalenosti si vytvorime samostatnou funkci a pouzijeme Haversinovu formuli, ktera vyuzije k vypoctu vzdalenosti udaje o langtitude a longtitude.

In [ ]:
bikes_data.head(1)

In [ ]:
# Create a fuction to calculate Distance between langtitude and longtitude both stations
def haversine_np(lon1,lat1,lon2,lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
# tvorba noveho sloupce "Distance in Km" a aplikace funkce haversine_np
bikes_data['distance in Km'] = round(haversine_np(bikes_data['s_lon'],bikes_data['s_lat'],bikes_data['e_lon'],bikes_data['e_lat']),1)

In [ ]:
bikes_data.head(1)

- po kontrole, kdy si vypiseme prvni radek vidime, ze nam pribyl sloupec 'distance in Km' a jako prvni vzdalenos je zapsano 0.9 Km, Id pocatecni stanice je 247 a jmeno Charlotte Spuare a Id konecne je 259 a jmeno St Andrew Squre. Delka vypujcky na tuto vzdalenost trvala 19.7 minut.
- nyni muzeme provest zakladni deskriptivni statistiku na sloupci 'distanece in Km'

In [ ]:
bikes_data['distance in Km'].describe()

In [ ]:
sort_distance = bikes_data.sort_values('distance in Km',ascending=False) # sort values from largest to smallest distance

In [ ]:
sort_distance

- prave jsem seradili vzdalenosti od nejvetsi po nejmensi. Muzeme si vsimnout ze nejvetsi vzdalenosti jsou pouze prvni tri hodnoty. A uz od prvniho pohledu nam tyto hodnoty nedavaji smysl. Muzeme to zduvodnit nekolika zpusoby:
- muzeme mit spatne udane hodnoty koncove Langtitude i Longtitude - ve svetle vsech udaju je mozne, ze zde doslo ke spatnemu zapisu longtitude pro konecnou stanici. Konstatuji to z toho duvodu, ze vsechny ostatni end_longtitude se nachazeji na hodnote -3 a dle google map hodnota, ktera se nachazi v teto dataframe znamena, ze konecna stanice se nachazi v centru Liverpoolu. Mozna by to bylo mozne, ale neodpovida tomu sloupec s hodnotou duration. Z toho duvodu tyto hodnoty odstranim a nebudu je brat v potaz.

- dale mame spoustu hodnot, kde distance je s hodnotou 0, ale mame zde durations, ktera neni v nulovych hodnotach. Proto si dovolim usoudit, ze na techto stanicich byly kola zapujceny a zaroven na stejne stanici i vraceny.

In [ ]:
distance = bikes_data[(bikes_data['distance in Km'] < 280)]
distance['distance in Km'].describe()

In [ ]:
max_distance = distance[(distance['distance in Km'] == distance['distance in Km'].max())]
max_distance

- ze statistiky nam vyplyva ze nejmensi hodnota vzdalenosti je 0 Km a maximalni vzdalenost je 18.4 Km. Prumerne to vychazi na 1.9 Km. Tam kde je vzdalenost 0 Km ale je zde uvedena doba trvani z meho pohledu se jedna o situaci, kdy se kolo vypujci i vrati ve stejne stanici( stanice se stejnym Id). Muzeme taky videt, mezi kterymi stanicemi je vzdalenost nejdelsi

In [ ]:
fig,axs = plt.subplots(4,1,figsize=(16,14))

ax1 = sns.histplot(data=distance,x='distance in Km',bins=20,ax = axs[0])
ax1.set_xlabel('Frequency Distance')

ax2 = sns.barplot(data=distance, x='weekday',y='distance in Km',ax = axs[1])
ax2.set_xticklabels(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"])
ax2.set_xlabel('Distance by weekday')

ax3 = sns.lineplot(data=distance,x='date',y='distance in Km',hue='date',ax=axs[2])
ax3.get_legend().remove()
ax3.set_xlabel('Distance by date')

ax4 = sns.barplot(data=distance, x='month',y='distance in Km',hue='weekday',ax=axs[3])
ax4.legend(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"],bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
ax4.set_xlabel('Distance by month')

plt.suptitle('Distance',va='center',fontsize=20,color='blue')
plt.subplots_adjust(hspace = 0.6)
plt.tight_layout()

plt.savefig('Distance_graf_png.png')

- ve vyse uvedenem grafu mame zobrazenou vzdalenost a jeji analyzu. Subplot_1 zobrazuje vzdalenost a mnozstvi vyskytu. Nejvyse casta vzdalenost je zhruba do 1Km, nasleduje vzdalenost do 2Km a postupne se snizuje. Subplot_2 nam zase vyjadruje vzdalenost v souvislosti na den v tydnu, muzeme si vsimnout ze o vikendu je vzdalenost nepatrne vetsi. Subplots_3 vyjadruje vyvoj vzdalenosti k datu pres cele sledovane obdobi. Subplots_4 je nakonec vzdalenost a jeji pohyb v prubehu mesicu a pro konkretni den v tydnu - letni mesice maji o vikendu (Sobota, Nedele) vetsi pocet vzdalenosti nez zimni mesice.

# Duration

- v teto casti analyzy se podivame na dobu trvani vypujcky. Uz na zacatku analyzy je upravena hodnota sloupce duration a to tak, ze je prepocitana na dobu vypujcky v minutach a ne sekundach.

In [ ]:
durations = bikes_data[(bikes_data['distance in Km'] < 280)]

- pro cast analyzy tykajici se doby trvani vypujcky jsem vyfiltrovala udaje , kde vzdalenost je mensi nez 280 km, protoze jsou zde predpokladany chybne udaje.

In [ ]:
durations['duration in min'].describe()

- z horni tabulky pro descriptivni statistiku muzeme ve vztahu k dobe vypujcky konstatovat, ze prumerna doba vypujcky je zhruba 32 minut, minimalni doba vypujcky je 1 minuta = zde bych se priklanela k tomu, ze byla vypujcka provedena ale hned nasledne taky zrusena. Maximalni doba vypujcky je hodne velka, takze by se to dalo vysvetlit tim, ze byla vypujcka provedena, ale clovek kolo vratil az po hodne dlouhe dobe.

In [ ]:
durations

In [ ]:
def change_width(ax, new_value):
    for patch in ax.patches:
        current_width = patch.get_width()
        diff = current_width - new_value
        
        # we change the bar width
        patch.set_width(new_value)
        
        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

In [ ]:
fig,axs = plt.subplots(3,1,figsize=(16,14))


ax1 = sns.histplot(data=durations,x='duration in min', bins=np.arange(1,121),
                 hue='month',alpha=.4,palette='rocket',ax=axs[0] )


ax2 = sns.barplot(data=durations,x='weekday',y='duration in min',
                 hue='weekday',palette='magma',errcolor='.5',dodge=False,ax=axs[1] )
ax2.set_xticklabels(["Mon","Tue","Wed","Thu","Fri","Sat","Sun"])
ax2.set_xlabel('Average bike share by weekday')

        
change_width(ax2, .35)

ax3 = sns.barplot(data=durations,x='season',y='duration in min',hue='season',dodge=False,ax=axs[2])
ax3.set_xlabel('Average bike share by season')
change_width(ax3, .40)


plt.suptitle('Duration bike share ',va='center',fontsize=20)
plt.subplots_adjust(wspace=.2,hspace=.2)
plt.tight_layout()

plt.savefig('Duration-Bike_Share.png')

- v hornich grafech mame zobrazenou dobu vypujcky. Hranici vypujcky jsem urcila v interaval od 1 minuty do 120 minut, protoze moc delsich  casovych intervalu vypujcky dataframe neobsahuje. Prumer vypujcky je kolem 30 minut. Delka trvani vypujcky je nejvyssi o vikendu v roce 2020 a 2019, ale je to ovlivneno tim, ze data pro rok 2018 a 2021 nejsou kompletni - zaznamy zacinaji na podzim v roce 2018 a konci v lete 2021. Z grafu muzeme vycist ze dlouha doba vypujcky je hlavne o vikendu, kde prevlada sobota a take v jarnich a letnich mesicich. V lete je prumer 40 minut na vypujcku nejmene je v zimnim obdobi a to kolem 25 minut na vypujcku. Nejcasteji se vyskytujici vypujcky v prubehu celeho roku trvaji 10 az 20 minut.

# Frekvence vypujcek podle casovych zaznamu

In [ ]:
bikes_data.head(1)

In [ ]:
f = bikes_data.groupby(['date','month']).agg({'start_id':'count'})
f.reset_index(inplace=True)

In [ ]:
f.rename(columns={'start_id':'start_counts'},inplace=True)

In [ ]:
f

In [ ]:
f.boxplot(by='month',column='start_counts',figsize=(12,8))

plt.show()

plt.savefig('Averige bike-share by month.png')

In [ ]:
f2 = bikes_data.groupby(['date','hour','weekday']).agg({'start_id':'count'})
f2.reset_index(inplace=True)
f2.rename(columns={'start_id':'start_counts'},inplace=True)

fig,axs = plt.subplots(2,1,figsize=(20,12))

ax1 = sns.barplot(x='hour',y='start_counts',hue='weekday',palette="husl",ci=None,data=f2,ax=axs[0])
ax2 = sns.lineplot(x='date',y='start_counts',data=f2,ax=axs[1])
ax1.set_ylabel('start stations counts')
ax2.set_ylabel('start stations counts')
change_width(ax1, .2)

plt.suptitle('Counts bike-share by date and hours',va='center',fontsize=20,color='blue')
plt.tight_layout()
plt.savefig('Counts bike-share by date and hours.png')

- na vrchnim grafu muzeme videt pocet vypujcek v jednotlivych hodinach pro jednotlive dny. Ve vsedni dny prevazuje mnozstvi vypujcek hlavne v ranich hodinach doba kolem 7 a 8 hodiny a odpolednich chodinach mezi 16 az 18, o vikendu je to v dopolednich hodinach od 10 hdiny do zhruba 15 hodiny odpoledne. Druhy graf znazornuje vyvoj vypujcek hodinove v prubehu celeho sledovaneho obdobi.

In [ ]:
rain_color = sns.color_palette('rainbow')[:4]  # tvorba vlastni barevne palety

f3 = bikes_data.groupby(['date','month','season','weekend','year','weekday']).agg({'start_id':'count'})
f3.reset_index(inplace=True)
f3.rename(columns={'start_id':'start_counts'},inplace=True)

In [ ]:
f3.head(1)

In [ ]:
fig,axes = plt.subplots(4,1,figsize=(12,14))

ax1 = sns.barplot(data=f3, x='season',y='start_counts',ci=False,palette=rain_color,ax=axes[0])

ax2 = sns.barplot(data=f3,x='month',y='start_counts',ci=False,palette=rain_color,ax=axes[1])
ax3 = sns.barplot(data=f3,x='weekend',y='start_counts',palette=rain_color,ax=axes[2])
ax3.set_xticklabels(['workday','weekend'])
ax3.set_xlabel('pracovni den a vikend')
ax4 = sns.boxplot(data=f3,x='weekday',y='start_counts',palette=rain_color,ax=axes[3])

plt.suptitle('counts bike-share by season,month,weekday and weekend',va='center',fontsize=20,color='blue')

fig.tight_layout()

plt.savefig('counts bike-share by season,month,weekday and weekend.png')

- vrchni grafy zobrazuji vypujcky v jednotlivych rocnich obdobich,jednotlive mesice a pro vikend nebo pracovni den. 

In [ ]:
weekdays_df = bikes_data.groupby('weekday').agg({'start_id':'count'})
weekdays_df.plot()
plt.show()
plt.savefig('bike-share for weekday.png')

- nejvyssi pocet vypujcek je v sobotu a nejnizsi v pondeli.

In [ ]:
Labels = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

fig,ax1 = plt.subplots(figsize=(14,8))

ax1.pie(weekdays_df.start_id,labels=Labels,autopct="%1.1f%%")
ax1.legend()

plt.suptitle('Percentil  bike-share by weekdays',va='center',fontsize=20,color='blue')
fig.tight_layout()
plt.show()

plt.savefig('Percentil bike-share by weekdays.png')

- ve vyse uvedenem grafu je znazornen procentualni podil pro kazdy den z celkoveho poctu vypujcek. Nejmene se bike share vyuzival v pondeli a nejvetsi procento vypujcek je v sobotu.

In [ ]:
by_date = bikes_data.pivot_table('start_id',aggfunc='count',index='date',columns='year')

In [ ]:
sns.set()
fig,axes = plt.subplots(4,1,figsize=(14,8))

sns.lineplot(data=by_date,x = 'date', y=by_date.iloc[:,0],color='black',linewidth=2.5, ax=axes[0])
sns.lineplot(data=by_date,x = 'date', y=by_date.iloc[:,1],color='r',linewidth=2.5, ax=axes[1])
sns.lineplot(data=by_date,x = 'date', y=by_date.iloc[:,2],color='g',linewidth=2.5, ax=axes[2])
sns.lineplot(data=by_date,x = 'date', y=by_date.iloc[:,3],color='b',linewidth=2.5, ax=axes[3])

plt.suptitle('counts by date',va='center',fontsize=20,color='blue')

fig.tight_layout()
plt.show()
plt.savefig('Counts by date and year.png')

- ctyri liniove grafy ukazujici vyvoj vypujcek v prubehu jednotlivych let a k datu. Uz bylo zmineno,ze rok 2018 a 2021 jsou nekompletni. Pro rok 2020 si muzeme vsimnout ze zde mame atakovanou hranici 2000 vypujcek za den, o rok drive byla hranice 1000 vypujcek za den. Lze i odhadnout mnozstvi vypujcek pro jednotlive rocni obdobi.

In [ ]:
counts_start_id = bikes_data['start_id'].value_counts()
counts_end_id = bikes_data['end_id'].value_counts()

- Id jednotlivych stanic a nastaveni Id jako indexu, vezmeme pouze unikatni hodnoty ID.

In [ ]:
counts_end_id.index

In [ ]:
counts_end_id.index

In [ ]:
s = bikes_data.copy()

In [ ]:
s.drop_duplicates('start_id',inplace=True)

In [ ]:
s.set_index('start_id',inplace=True)

In [ ]:
start_df = pd.merge(s,counts_start_id, left_index=True, right_index=True)

In [ ]:
start_df.rename(columns={'start_id':'start_counts'},inplace=True)

In [ ]:
sort_start_id = start_df.sort_values(by='start_counts',ascending=False)

In [ ]:
sort_start_id.head(1)

In [ ]:
sort_start_id.reset_index(inplace=True)

In [ ]:
sort_start_id.rename(columns={'level_0':'start_stations_Id'},inplace=True)

In [ ]:
sort_start_id['start_stations_Id'] = sort_start_id['start_stations_Id'].astype(str)

In [ ]:
sort_start_id['id_name'] = sort_start_id['start_stations_Id'] + ' ' + sort_start_id['start_name']

In [ ]:
e = bikes_data.copy()

In [ ]:
e.drop_duplicates('end_id',inplace=True)

In [ ]:
e.set_index('end_id',inplace=True)

In [ ]:
end_df = pd.merge(e,counts_end_id, left_index=True, right_index=True)

In [ ]:
end_df.rename(columns={'end_id':'end_counts'},inplace=True)

In [ ]:
sort_end_id = end_df.sort_values(by='end_counts',ascending=False)

In [ ]:
sort_end_id.reset_index(inplace=True)

In [ ]:
sort_end_id.rename(columns={'level_0':'end_stations_Id'},inplace=True)

In [ ]:
sort_end_id['end_stations_Id'] = sort_end_id['end_stations_Id'].astype(str)

In [ ]:
sort_end_id['id_name'] = sort_end_id['end_stations_Id'] + ' ' + sort_end_id['end_name']

In [ ]:
sort_end_id.head(1)

In [ ]:
sns.set(font_scale=1.2)
fig,axs = plt.subplots(2,2,figsize=(18,18))



ax1 = sns.barplot(data=sort_start_id,x='start_counts',y=sort_start_id['id_name'][:20],ax=axs[0][0])

ax2 = sns.barplot(data=sort_start_id,x='start_counts',y=sort_start_id['id_name'][-20:],palette='viridis',ax=axs[0][1])
ax3 = sns.barplot(data=sort_end_id,x='end_counts',y=sort_end_id['id_name'][:20],ax=axs[1][0])
ax4 = sns.barplot(data=sort_end_id,x='end_counts',y=sort_end_id['id_name'][-20:],palette='viridis',ax=axs[1][1])

ax1.set_xlabel(xlabel="Maximal frequently used Started_Station",fontsize=15)
ax1.set_ylabel(ylabel="started_Station",fontsize=15)
    

ax2.set_xlabel(xlabel="Minimal frequently used Started_Station",fontsize=15)
ax2.set_ylabel(ylabel="started_Station",fontsize=15)

ax3.set_xlabel(xlabel="Maximal frequently used Ended_Station",fontsize=15)
ax3.set_ylabel(ylabel="ended_Station",fontsize=15)

ax4.set_xlabel(xlabel="Minimal frequently used Ended_Station",fontsize=15)
ax4.set_ylabel(ylabel="ended_Station",fontsize=15)

plt.suptitle('Maximal and Minimal Frequently of Started and Ended Station',va='center',fontsize=20)
plt.subplots_adjust(wspace=.2,hspace=.2)
plt.tight_layout()
plt.show()
plt.savefig('Maximal and minimal frequence for start and end stations.png')

- ve vyse uvedenych grafech mame zobrazene nejvice a nejmene vyuzivane stanice a to jak pocatecni tak konecny.

In [ ]:
start_describe_statistic = sort_start_id['start_counts']
start_describe_statistic.describe()

- descriptivni statistika pro pocatecni stanice nam rika, ze mame celkove 198 stanic s jejich unikatnim ID, nektera stanice byla vyuzivana pouze jednout, maximalni pocet 17390.

In [ ]:
end_describe_statistic = sort_end_id['end_counts']
end_describe_statistic.describe()

- descriptivni statistika pro konecne stanice nam rika, ze mame celkove 199 stanic s jejich unikatnim ID, nektera stanice byla vyuzivana pouze dvakrat, maximalni pocet 16656.

## Weather and Bike share

In [ ]:
weather_data.head(1)

In [ ]:
# season
season = (weather_data['date'].dt.month-1)//3
season += (weather_data['date'].dt.month == 3)&(weather_data['date'].dt.day>=20)
season += (weather_data['date'].dt.month == 6)&(weather_data['date'].dt.day>=21)
season += (weather_data['date'].dt.month == 9)&(weather_data['date'].dt.day>=23)
season -= 3*((weather_data['date'].dt.month == 12)&(weather_data['date'].dt.day>=21)).astype(int)
weather_data['season'] = season

In [ ]:
weather_data['month'] = weather_data['date'].dt.month
weather_data['year'] = weather_data['date'].dt.year

In [ ]:
weather_data.dtypes

In [ ]:
weather = weather_data.groupby(['date','season']).agg({'temp':'mean',
                                           'wind':'mean',
                                           'rain':'mean',
                                           'humidity':'mean',
                                           'cloud':'mean'})

In [ ]:
weather.reset_index(inplace=True)

In [ ]:
weather_2 = weather[(weather['date']>='2018-09-15')]

In [ ]:
bike = bikes_data.groupby('date').agg({'start_id':'count',
                                      'duration in min':'mean',
                                      'distance in Km':'mean'})

In [ ]:
bike.reset_index(inplace=True)

In [ ]:
bike_2 = bike[(bike['date']<='2020-10-31')]

In [ ]:
df = bike_2.merge(weather_2, on='date',how='left')

In [ ]:
df.rename(columns={'start_id':'stations_counts'},inplace=True)

In [ ]:
df['stations_counts'].max() # maximalni pocet vypujcek za jeden den

In [ ]:
df['stations_counts'].min() # minimalni pocet vypujcek za jeden den

In [ ]:
df['weekday'] = df['date'].dt.weekday
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
df[df['stations_counts'].max() == df['stations_counts']]

In [ ]:
df[df['stations_counts'].min() == df['stations_counts']]

- vyse uvedeny radek nam rika, ze nejvyssi pocet vyujcek probehl dne 20.6.2020, prumerna doba trvani byla 46 minut, vzdalenost prumerne 2.5 km a prumernou teplotu, vitr, dest, vlhkost, oblacnost. Den byla sobota a pocet byl 2428.
- nejnizsi pocet vypujcek probehl 15.9.2018 pouze 3, vzdalenost 1,3km a prumerna doba 30 minut.

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
df1 = df[['stations_counts','duration in min','distance in Km','temp','wind','rain','humidity','cloud','weekday']]

plt.figure(figsize=(16,6))
heatmap = sns.heatmap(df1.corr(),vmin=-1,vmax=1,annot=True)

plt.suptitle('Correlation bike-share and weather',va='center',fontsize=20)

plt.show()
plt.savefig('Correlation bike-share and weather.png')

- vyse uvedeny korelacni graf ukazuje tesnost zmeny hodnoty znaku zavisle promenne pri zmene hodnoty nezavisle promenne. K vypoctu korelacniho poctu byla aplikovana metoda corr() na pomocne dataframe, ktera vznikla spojenim puvodnich dvou datasframe.

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(16,20))
ax1 = sns.regplot(data=df1,y='stations_counts',x='temp',scatter_kws={'s':100,'alpha':0.5},ax=axs[0][0])
ax1 = sns.regplot(data=df1,y='stations_counts',x='wind',marker='+',scatter_kws={'s':100,'alpha':0.5},ax=axs[0][1])
ax2 = sns.regplot(data=df1,y='stations_counts',x='rain',marker='x',scatter_kws={'s':100,'alpha':0.5},ax=axs[0][2])

ax3 = sns.regplot(data=df1,y='duration in min',x='temp',scatter_kws={'s':100,'alpha':0.5},color='r',ax=axs[1][0])
ax4 = sns.regplot(data=df1,y='duration in min',x='wind',color='r',marker='+',scatter_kws={'s':100,'alpha':0.5},ax=axs[1][1])
ax5 = sns.regplot(data=df1,y='duration in min',x='rain',color='r',marker='x',scatter_kws={'s':100,'alpha':0.5},ax=axs[1][2])

ax6 = sns.regplot(data=df1,y='distance in Km',x='temp',color='g',scatter_kws={'s':100,'alpha':0.5},ax=axs[2][0])
ax7 = sns.regplot(data=df1,y='distance in Km',x='wind',color='g',marker='+',scatter_kws={'s':100,'alpha':0.5},ax=axs[2][1])
ax8 = sns.regplot(data=df1,y='distance in Km',x='rain',color='g',marker='x',scatter_kws={'s':100,'alpha':0.5},ax=axs[2][2])

plt.suptitle('Linear regression bike-share and weather - part1',va='center',fontsize=20)

plt.tight_layout()
plt.show()
plt.savefig('Relationship between bikeshare and weather - part1.png')

- vyse uvedene grafy linearni regrese nam ukazuji zavislost mezi velicinamy. V kazdem grafu je znazornena jako velicina x jedna nase promenna napr.teplota a jako velicina y druha promena napr.distance. V nasem pripade muzeme rict, ze pravdepodobne na teplote je zavisla distance a ne naopak. Stejne tak to vypovida graf, ktery se tyka doby trvani vypujcky, ze ktereho muzeme vcelku zato rict, ze tam mame primou zavislost - hodnoty se vyskytuji podel primky. Kdezto u poctu vypujcek jsou hodnoty vice rozsete, a tak lze konstatovat, ze pocet vypujcek neni uplne zavisly na tom, jake je pocasi.

In [ ]:
sns.set(rc={'figure.figsize':(16,14),'legend.fontsize':30,'legend.title_fontsize':30,'axes.labelsize':24,
           'xtick.labelsize':22,'ytick.labelsize':22})
g = sns.pairplot(df1,vars=['stations_counts','duration in min','distance in Km',
                           'humidity','cloud'],hue='weekday',corner=True,diag_kind='hist',height=6,
                plot_kws={'s':20})

plt.suptitle('Relationship between bikeshare and weather - part 2',va='center',fontsize=20)
plt.show()
plt.savefig('Relationship betwen bikeshare and weather - part2.png')